## 1 读写文本格式的数据

### 1.1 read_csv/table

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.read_csv('ch06/f1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
pd.read_table('ch06/f1.csv', sep=',')

In [ ]:
pd.read_csv('ch06/f2_无标题行.csv', header=None)

In [ ]:
pd.read_csv('ch06/f2_无标题行.csv', names=['a','b','c','d','message'])

In [ ]:
# 将message列做成DataFrame的索引
pd.read_csv('ch06/f2_无标题行.csv', names=['a','b','c','d','message'], index_col='message')

In [ ]:
pd.read_csv('ch06/f3.csv', index_col=['key1', 'key2'])

In [ ]:
list(open('ch06/f4.txt'))

In [ ]:
pd.read_table('ch06/f4.txt', sep='\s+')  # 利用正则表达式\s+匹配任何空白字符，包括空格、制表符、换页符等等

In [ ]:
pd.read_table('ch06/f4.txt', sep='\s+', skiprows=[1,2])

### 1.2 创建文件并写入DataFrame(只能用.to_csv)/ List数据..

In [ ]:
# 方法一：利用pandas、os
import pandas as pd
import numpy as np
import os

path = 'ch06/f5_os.csv'
df = pd.DataFrame(np.random.randn(10000,4), columns=['one','two','three','four'])
df2 = pd.DataFrame((chr(np.random.randint(65,90)) for i in range(10000)), columns=['key'])
df = pd.concat([df, df2], axis=1)
# pd.concat([df,df2], axis=1)

df.to_csv(path, index=False, mode='a')
# 文件不存在则写入header，否则则将df写入
# if not os.path.exists(path):
#    df.to_csv(path, header=['one','two','three','four','key'], index=False, mode='a')
# else:
#    df.to_csv(path, header=False, index=False, mode='a')

In [ ]:
# 方法二：利用csv
import numpy as np
import csv

path2 = 'ch06/f6_csv.csv'
file = open(path2, 'a+', encoding='utf-8', newline='')

csv_writer = csv.writer(file)
csv_writer.writerow([f'one','two','three'])
csv_writer.writerows([[1,'luke','96'],[2,'jack','85'],[3,'nick','84']])

file.close()

In [ ]:
# 逐行读取文件
pd.read_csv('ch06/f5_os.csv', nrows=5)

In [ ]:
# 逐块读取文件
chunker = pd.read_csv('ch06/f5_os.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    # piece is a DataFrame,lenth is chunksize=1000,and piece[key] is a Series ,key is int ,value is the key column
#     print(piece['key'].value_counts())
    # "piece[key] value_counts" is a Series ,key is the key column, and value is the key count
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
#     chunkercount+=1

tot = tot.sort_values(ascending=False)
tot[:10]

### 1.3 将数据写出到文本格式 df.to_csv('sys.stdout', args)

In [ ]:
import sys
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame(np.random.randn(4,3), columns=['one','two','three'])
data.to_csv('ch06/f7.csv')

In [ ]:
pd.read_csv('ch06/f7.csv')

In [ ]:
data.to_csv(sys.stdout)

In [ ]:
data.to_csv(sys.stdout, sep='|')  # 对data在文件中的数据做|匹配后重定向后打印出来，不改变原数据

In [ ]:
data2 = pd.read_csv('ch06/f8.csv')
# data2.to_csv(sys.stdout)
data2

In [ ]:
data2.to_csv(sys.stdout, na_rep='NULL')

In [ ]:
data2.to_csv(sys.stdout, index=False, header=False)

In [ ]:
data2.to_csv(sys.stdout, index=False, columns=['a','b','c'])

In [ ]:
# Series写入文本
dates = pd.date_range('1/1/2023', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('ch06/f9_tseries.csv')

In [ ]:
# pd.read_csv('ch06/f9_tseries.csv')

In [ ]:
ts.to_csv(sys.stdout)

### 1.4 处理分隔符格式

In [ ]:
data2 = pd.read_csv('ch06/f10.csv')
data2.to_csv(sys.stdout)

In [1]:
import csv
f = open('ch06/f10.csv')
reader = csv.reader(f)
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [ ]:
import csv
# 1.读取文件到一个多行列表；此次利用with打开文件，则with外的其他地方读不到f
with open('ch06/f10.csv') as f:
    lines = list(csv.reader(f))

In [ ]:
# 2.将⾏分为标题⾏和数据⾏
header, values = lines[0], lines

In [ ]:
# 3.字典构造式和zip（*value），将行转置换为列
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [ ]:
# 4_1. 定义csv.Dialect的一个子类
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
f2 = open('ch06/f10.csv')
reader = csv.reader(f2, dialect=my_dialect)

In [ ]:
# 4_2. 直接参数写明，不写子类
f2 = open('ch06/f10.csv')
reader = csv.reader(f2, delimiter='|')

In [ ]:
for line in reader:
    print(line)

In [ ]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### 1.5 JSON数据

In [ ]:
import json
import pandas as pd

In [ ]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": ["Sixes", "Stache", "Cisco"]}]
}"""
# siblings兄弟姐妹

result = json.loads(obj)
result

In [ ]:
# json.dumps则将Python对象转换成JSON格式
json.dumps(result)

In [ ]:
# JSON对象转换为DataFrame：向DataFrame构造器传入一个字典的列表
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

In [ ]:
tables = pd.read_html('ch06/fdic_failed_bank_list.html')

In [ ]:
len(tables)

In [ ]:
failures = tables[0]
failures.head()

In [ ]:
# 按年份计算倒闭的银⾏数
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

### 1.6 利用lxml.objectify解析XML

In [ ]:
from lxml import objectify
import pandas as pd

In [ ]:
path = 'ch06/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [ ]:
# root.INDICATOR返回⼀个⽤于产⽣各个<INDICATOR>XML元素的⽣成器。
data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR_UNIT:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = pd.DataFrame(data)
perf.head()

In [ ]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [ ]:
root

In [ ]:
root.get('href')

In [ ]:
root.text

## 2 二进制数据格式

### 2.1 使用HDF5格式

In [2]:
import pandas as pd
import numpy as np

In [3]:
frame = pd.read_csv('ch06/f1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
frame.to_pickle('ch06/frame_pickle')

In [ ]:
pd.read_pickle('ch06/frame_pickle')

In [ ]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [ ]:
store = pd.HDFStore('ch06/mydata.h5')

In [ ]:
# 添加数据：way1
store['obj1'] = frame

In [ ]:
store['obj1_col'] = frame['a']

In [ ]:
store

In [ ]:
# 查询/ 读取数据
store['obj1']

In [ ]:
# way2
store.put('obj2', frame, format='table')

In [ ]:
store.select('obj2', where=['index >= 10 and index <= 15'])

In [ ]:
store.close()

In [ ]:
# way3
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [ ]:
pd.read_hdf('mydata.h5','obj3', where=['index < 5']) 

### 2.2 读取Microsoft Excel文件

In [ ]:
import pandas as pd

In [ ]:
# ExcelFile + read_excel：读取excel文件
xlsx = pd.ExcelFile('ch06/ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

In [ ]:
frame = pd.read_excel('ch06/ex1.xlsx', 'Sheet1')
frame

In [ ]:
# ExcelWriter + to_excel：写入Excel文件
writer = pd.ExcelWriter('ch06/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')  # frame.to_excel('examples/ex2.xlsx')

In [ ]:
writer.save()

### 2.3 Web APIs交互

In [ ]:
import requests

In [ ]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [ ]:
resp = requests.get(url)
resp

In [ ]:
data = resp.json()

In [ ]:
data[0]['title']

In [ ]:
issues = pd.DataFrame(data, columns=['number', 'title ', 'labels', 'state'])
issues

### 2.4 数据库交互

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# 该API打开一个到 SQLite数据库文件 database的链接
con = sqlite3.connect('mydata.sqlite')

In [ ]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);"""

# 执行sql语句，可以用占位符？表示参数
con.execute(query)

In [ ]:
# 提交当前事务，所做的动作对数据库连接才可见 
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
('Tallahassee', 'Florida', 2.6, 3),
('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

# 对data执行一个sql命令（stmt）
con.executemany(stmt, data)

In [ ]:
cursor = con.execute('select * from test')

In [ ]:
rows = cursor.fetchall()  # 获取查询结果集中所有（剩余）的行，返回一个列表

In [ ]:
rows

In [ ]:
cursor.description

In [ ]:
# 元组列表传给DataFrame构造器，但还需要列名（位于光标的description属性中）
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [ ]:
import sqlalchemy as sqla
import pandas as pd

In [ ]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [ ]:
pd.read_sql('select * from test', db)

#### 连接mysql并创建表添加数据

In [1]:
# python连接mysql，有2种方式，分别是利用sqlalchemy、pymysql这2个包连接
import pandas as pd
import numpy as np
# from sqlalchemy import create_engine
import pymysql

In [3]:
# 创建数据库连接
connect = pymysql.connect(
    host = 'localhost',
    db = 'cy01',
    user = 'root',
    passwd = '123456',
    charset = 'utf8'
    # use_unicode = True
)

# 使用cursor()方法创建一个游标对象，然后赋值给cur
cur = connect.cursor()

In [ ]:
# 将存在的表删除
sql_drop = """
    DROP TABLE IF EXISTS from_python"""

cur.execute(sql_drop)
connect.commit()  # 提交到数据库执行；事务代码

In [ ]:
# 创建数据库表
sql_create = """
    CREATE TABLE from_python(id INTEGER(10) PRIMARY KEY, name VARCHAR(20));
"""
cur.execute(sql_create)
connect.commit()
connect.close()

In [ ]:
help(pymysql.connect)

In [ ]:
# 插入数据
# sql = "INSERT INTO EMPLOYEE(FIRST_NAME, \
#        LAST_NAME, AGE, SEX, INCOME) \
#        VALUES (%s, %s, %s, %s, %s )" % \
#        ('Mac', 'Mohan', 20, 'M', 2000)
sql = """INSERT INTO from_python(id, name) VALUES(1, 'wren')"""
try:
    # 执行sql语句
    cur.execute(sql)
    # 提交到数据库执行
    connect.commit()
except:
    connect.rollback()

# 关闭数据库连接
connect.close()

In [5]:
sql = "SELECT * FROM from_python;"
cur.execute(sql)

# 获取所有记录列表
results = cur.fetchall()
results

((1, 'wren'),)